In [1]:
#For the import of packages
import numpy as np
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
import qiime2 as q2

data_dir = "data/"

In [2]:
#data import
! wget -nv -O $data_dir/sequences_demux_paired.qza 'https://polybox.ethz.ch/index.php/s/KEQxsekRow6lOAz/download'
! wget -nv -O $data_dir/metadata.tsv 'https://polybox.ethz.ch/index.php/s/RVPyoVYUBeVQFrJ/download'

2022-10-10 16:42:57 URL:https://polybox.ethz.ch/index.php/s/KEQxsekRow6lOAz/download [8529260946/8529260946] -> "data//sequences_demux_paired.qza" [1]
2022-10-10 16:43:01 URL:https://polybox.ethz.ch/index.php/s/RVPyoVYUBeVQFrJ/download [325995/325995] -> "data//metadata.tsv" [1]


In [2]:
df_meta = pd.read_csv('data/metadata.tsv', sep='\t')
df_meta

,id,Library Layout,Instrument,collection_date,geo_location_name,geo_latitude,geo_longitude,host_id,age_days,weight_kg,...,birth_length_cm,sex,delivery_mode,zygosity,race,ethnicity,delivery_preterm,diet_milk,diet_weaning,age_months
0,ERR1314182,PAIRED,Illumina MiSeq,2011-11-11 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,42.1,232.0,NaN,...,47.0,male,Cesarean,Monozygotic,Caucasian,Not Hispanic,True,fd,True,8.0
1,ERR1314183,PAIRED,Illumina MiSeq,2010-12-11 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,27.2,192.0,NaN,...,45.0,female,Cesarean,Dizygotic,Caucasian,Hispanic,True,fd,True,6.0
2,ERR1314184,PAIRED,Illumina MiSeq,2011-12-11 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,28.1,536.0,NaN,...,51.0,female,Cesarean,Monozygotic,Caucasian,Not Hispanic,False,NaN,NaN,18.0
3,ERR1314185,PAIRED,Illumina MiSeq,2011-12-11 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,28.2,537.0,NaN,...,50.0,female,Cesarean,Monozygotic,Caucasian,Not Hispanic,False,NaN,NaN,18.0
4,ERR1314186,PAIRED,Illumina MiSeq,2013-01-12 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,39.2,688.0,NaN,...,48.0,male,Cesarean,Monozygotic,African-American,Not Hispanic,True,NaN,NaN,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1664,ERR1310705,PAIRED,Illumina MiSeq,2010-07-31 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,26.1,74.0,NaN,...,49.0,female,Vaginal,Dizygotic,Caucasian,Not Hispanic,False,fd,False,2.0
1665,ERR1310707,PAIRED,Illumina MiSeq,2011-02-07 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,27.1,250.0,NaN,...,45.0,female,Cesarean,Dizygotic,Caucasian,Hispanic,True,fd,True,8.0
1666,ERR1310708,PAIRED,Illumina MiSeq,2011-04-09 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,27.1,310.0,NaN,...,45.0,female,Cesarean,Dizygotic,Caucasian,Hispanic,True,fd,True,10.0
1667,ERR1310709,PAIRED,Illumina MiSeq,2012-01-18 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,27.1,594.0,NaN,...,45.0,female,Cesarean,Dizygotic,Caucasian,Hispanic,True,NaN,NaN,20.0


In [10]:
#first lose all host_id duplicates
#here this is not problematic because the columns of interest (preterm. delivery_mode, birth weight etc.) are all the same for each host_id
df_meta_red = df_meta.drop_duplicates(subset=['host_id'])
df_meta_red.to_csv('df_meta_noduplicates', sep='')

,id,Library Layout,Instrument,collection_date,geo_location_name,geo_latitude,geo_longitude,host_id,age_days,weight_kg,...,birth_length_cm,sex,delivery_mode,zygosity,race,ethnicity,delivery_preterm,diet_milk,diet_weaning,age_months
0,ERR1314182,PAIRED,Illumina MiSeq,2011-11-11 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,42.1,232.0,NaN,...,47.0,male,Cesarean,Monozygotic,Caucasian,Not Hispanic,True,fd,True,8.0
1,ERR1314183,PAIRED,Illumina MiSeq,2010-12-11 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,27.2,192.0,NaN,...,45.0,female,Cesarean,Dizygotic,Caucasian,Hispanic,True,fd,True,6.0
2,ERR1314184,PAIRED,Illumina MiSeq,2011-12-11 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,28.1,536.0,NaN,...,51.0,female,Cesarean,Monozygotic,Caucasian,Not Hispanic,False,NaN,NaN,18.0
3,ERR1314185,PAIRED,Illumina MiSeq,2011-12-11 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,28.2,537.0,NaN,...,50.0,female,Cesarean,Monozygotic,Caucasian,Not Hispanic,False,NaN,NaN,18.0
4,ERR1314186,PAIRED,Illumina MiSeq,2013-01-12 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,39.2,688.0,NaN,...,48.0,male,Cesarean,Monozygotic,African-American,Not Hispanic,True,NaN,NaN,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,ERR1310645,PAIRED,Illumina MiSeq,2010-10-23 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,19.1,182.0,6.917,...,47.0,male,Cesarean,Monozygotic,Caucasian,Not Hispanic,True,bd,True,6.0
946,ERR1310554,PAIRED,Illumina MiSeq,2012-04-14 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,24.2,710.0,13.353,...,48.0,male,Cesarean,Dizygotic,Caucasian,Not Hispanic,False,NaN,NaN,23.0
956,ERR1310464,PAIRED,Illumina MiSeq,2010-09-03 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,11.2,156.0,NaN,...,50.0,male,Vaginal,Dizygotic,Caucasian,Not Hispanic,False,fd,True,5.0
995,ERR1310538,PAIRED,Illumina MiSeq,2011-07-19 00:00:00,"USA, Missouri, St. Louis",38.63699,-90.263794,24.1,441.0,11.751,...,48.0,male,Cesarean,Dizygotic,Caucasian,Not Hispanic,False,NaN,NaN,14.0


1. Differences in birth weight in preterm and non-preterm delivery babies

In [12]:
preterm = df_meta_red[df_meta_red['delivery_preterm'] == True]
notpreterm = df_meta_red[df_meta_red['delivery_preterm'] == False]
#calculate average
preterm_weight = preterm['birth_weight_g'].mean()
notpreterm_weight = notpreterm['birth_weight_g'].mean()

print('Preterm average weight:', preterm_weight)
print('Non-preterm average weight:', notpreterm_weight)
print('Difference:', preterm_weight - notpreterm_weight)

Preterm average weight: 2404.6190476190477
Non-preterm average weight: 2813.9473684210525
Difference: -409.3283208020048


2. Differences in birth height in preterm and non-preterm delivery babies

In [13]:
#calculate average
preterm_length = preterm['birth_length_cm'].mean()
notpreterm_length = notpreterm['birth_length_cm'].mean()

print('Preterm average length:', preterm_length)
print('Non-preterm average length:', notpreterm_length)
print('Difference:', preterm_length - notpreterm_length)

Preterm average length: 47.19047619047619
Non-preterm average length: 49.36842105263158
Difference: -2.1779448621553925


3. Preterm delivery and delivery mode

In [16]:
ces = df_meta_red[df_meta_red['delivery_mode'] == 'Cesarean']
vag = df_meta_red[df_meta_red['delivery_mode'] == 'Vaginal']
emerg = df_meta_red[df_meta_red['delivery_mode'] == 'Cesarean_emergency']

ces_tot = ces['id'].nunique()
vag_tot = vag['id'].nunique()
emerg_tot = emerg['id'].nunique()

ces_pre = ces['delivery_preterm'].sum()
vag_pre = vag['delivery_preterm'].sum()
emerg_pre = emerg['delivery_preterm'].sum()

print('Percentage of preterm delivery in different delivery modes:')
print('Cesarean:', (ces_pre/ces_tot)*100)
print('Vaginal:', (vag_pre/vag_tot)*100)
print('Cesarean Emergency:', (emerg_pre/emerg_tot)*100)

Percentage of preterm delivery in different delivery modes:
Cesarean: 58.333333333333336
Vaginal: 41.37931034482759
Cesarean Emergency: 66.66666666666666
